In [1]:
val orders_schema = "order_id int, order_date string, customer_id int,order_status string"

orders_schema = order_id int, order_date string, customer_id int,order_status string


order_id int, order_date string, customer_id int,order_status string

In [2]:
val orders_df = spark.read.format("csv").schema(orders_schema).load("/public/trendytech/orders/orders_1gb.csv")

Waiting for a Spark session to start...

orders_df = [order_id: int, order_date: string ... 2 more fields]


[order_id: int, order_date: string ... 2 more fields]

In [3]:
orders_df.createOrReplaceTempView("orders")

In [4]:
spark.sql("""select order_id*1 as new_order_id, order_status from
(select order_id, customer_id, order_status from orders where order_id < 500)
where order_id < 200""").show()

+------------+---------------+
|new_order_id|   order_status|
+------------+---------------+
|           1|         CLOSED|
|           2|PENDING_PAYMENT|
|           3|       COMPLETE|
|           4|         CLOSED|
|           5|       COMPLETE|
|           6|       COMPLETE|
|           7|       COMPLETE|
|           8|     PROCESSING|
|           9|PENDING_PAYMENT|
|          10|PENDING_PAYMENT|
|          11| PAYMENT_REVIEW|
|          12|         CLOSED|
|          13|PENDING_PAYMENT|
|          14|     PROCESSING|
|          15|       COMPLETE|
|          16|PENDING_PAYMENT|
|          17|       COMPLETE|
|          18|         CLOSED|
|          19|PENDING_PAYMENT|
|          20|     PROCESSING|
+------------+---------------+
only showing top 20 rows



In [5]:
spark.sql("""select order_id*1 as new_order_id, order_status from
(select order_id, customer_id, order_status from orders where order_id < 500)
where order_id < 200""").explain(true)

== Parsed Logical Plan ==
'Project [('order_id * 1) AS new_order_id#20, 'order_status]
+- 'Filter ('order_id < 200)
   +- 'SubqueryAlias `__auto_generated_subquery_name`
      +- 'Project ['order_id, 'customer_id, 'order_status]
         +- 'Filter ('order_id < 500)
            +- 'UnresolvedRelation `orders`

== Analyzed Logical Plan ==
new_order_id: int, order_status: string
Project [(order_id#0 * 1) AS new_order_id#20, order_status#3]
+- Filter (order_id#0 < 200)
   +- SubqueryAlias `__auto_generated_subquery_name`
      +- Project [order_id#0, customer_id#2, order_status#3]
         +- Filter (order_id#0 < 500)
            +- SubqueryAlias `orders`
               +- Relation[order_id#0,order_date#1,customer_id#2,order_status#3] csv

== Optimized Logical Plan ==
Project [(order_id#0 * 1) AS new_order_id#20, order_status#3]
+- Filter ((isnotnull(order_id#0) && (order_id#0 < 500)) && (order_id#0 < 200))
   +- Relation[order_id#0,order_date#1,customer_id#2,order_status#3] csv

== Physi

In [6]:
import org.apache.spark.sql.catalyst.expressions.Literal
import org.apache.spark.sql.catalyst.rules.Rule
import org.apache.spark.sql.catalyst.plans.logical.LogicalPlan
import org.apache.spark.sql.catalyst.expressions._

In [7]:
object MultiplyOptimizationRule extends Rule[LogicalPlan] {
def apply(plan: LogicalPlan): LogicalPlan = plan transformAllExpressions {
case Multiply(left,right) if right.isInstanceOf[Literal] &&
right.asInstanceOf[Literal].value.asInstanceOf[Integer] == 1 =>
println("optimization of one applied")
left
}
}

defined object MultiplyOptimizationRule


In [8]:
spark.experimental.extraOptimizations = Seq(MultiplyOptimizationRule)

spark.experimental.extraOptimizations: Seq[org.apache.spark.sql.catalyst.rules.Rule[org.apache.spark.sql.catalyst.plans.logical.LogicalPlan]] = List(MultiplyOptimizationRule$@103a5ac6)


In [9]:
spark.sql("""select order_id*1 as new_order_id, order_status from
(select order_id, customer_id, order_status from orders where order_id < 500)
where order_id < 200""").explain(true)

optimization of one applied
== Parsed Logical Plan ==
'Project [('order_id * 1) AS new_order_id#24, 'order_status]
+- 'Filter ('order_id < 200)
   +- 'SubqueryAlias `__auto_generated_subquery_name`
      +- 'Project ['order_id, 'customer_id, 'order_status]
         +- 'Filter ('order_id < 500)
            +- 'UnresolvedRelation `orders`

== Analyzed Logical Plan ==
new_order_id: int, order_status: string
Project [(order_id#0 * 1) AS new_order_id#24, order_status#3]
+- Filter (order_id#0 < 200)
   +- SubqueryAlias `__auto_generated_subquery_name`
      +- Project [order_id#0, customer_id#2, order_status#3]
         +- Filter (order_id#0 < 500)
            +- SubqueryAlias `orders`
               +- Relation[order_id#0,order_date#1,customer_id#2,order_status#3] csv

== Optimized Logical Plan ==
Project [order_id#0 AS new_order_id#24, order_status#3]
+- Filter ((isnotnull(order_id#0) && (order_id#0 < 500)) && (order_id#0 < 200))
   +- Relation[order_id#0,order_date#1,customer_id#2,order_s